# The Inflation Floor: Why Disinflation Is Over  
*Lighthouse Macro — 2025-09-28*

> TL;DR — The disinflationary tailwind of 1990–2019 is gone. Four structural forces
> (demographics, deglobalization, climate/energy capex, fiscal dominance) are
> raising the **inflation floor**. Markets are still priced for the old world.

---

## 1) Setting the Stage: Inflation’s Comeback, Policy’s Dilemma

Tactically, inflation rolled off the 2022 peaks. Strategically, **structural forces**
are preventing a full reversion to the pre-COVID “lowflation” regime. Monetary policy
can crush demand; it cannot print oil, copper, workers, or grid capacity.

![CPI vs Fed Funds](charts/01_cpi_vs_fedfunds.png)

---

## 2) Markets Are Pricing a Mild Future — Maybe Too Mild

Breakevens say future inflation will be contained. But breakevens embed liquidity,
risk premia, and Treasury supply dynamics; they are not a pure crystal ball.

![Breakevens vs CPI](charts/02_breakevens_vs_cpi.png)

The **5y5y** remains anchored — a victory for central bank credibility, or complacency?

![5y5y Inflation Expectations](charts/03_5y5y.png)

---

## 3) The Term Structure Is Screaming “Regime Shift”

The yield curve inversion is not just a growth signal; it’s a **term premia** story as
well. When deficits are large and QT is ongoing, the long end demands compensation.

![Treasury Yields & Curve](charts/04_yield_curve.png)

---

## 4) Dollar, Real Yields, and the Global Liquidity Map

A firm dollar + higher real yields tightens global financial conditions. In a world with
fragmented liquidity cycles, domestic policy bites globally.

![Dollar vs Real Yields](charts/05_dxy_vs_real10y.png)

---

## 5) Commodities as Policy Thermometers

Oil is the swing factor; copper is the bottleneck for electrification. Together, they
are policy thermometers for the inflation floor.

![Oil & Copper vs CPI](charts/06_oil_copper_vs_cpi.png)

---

## 6) Plumbing Still Matters: RRP, SOFR, and the Price of Money

Even in a “big picture” view, the **plumbing** leaks into macro outcomes. The ON RRP
and SOFR tell you where secured funding stands and how quickly the policy impulse hits.

![RRP & SOFR vs CPI](charts/07_rrp_sofr_vs_cpi.png)

---

## 7) Reindustrialization Is Real — and Inflationary at the Margin

U.S. manufacturing construction is in a capex boom. Reshoring and industrial policy are
supportive of a higher inflation floor (after a lag).

![Manufacturing Construction](charts/08_manufacturing_construction.png)

Durable goods orders are off the 2020 slump and point to cyclical resilience.

![Durable Goods Orders](charts/09_durable_goods_orders.png)

---

## 8) Labor: Tight Enough to Be Sticky

The unemployment rate remains low relative to **job openings**. Wage growth won’t revert
to 2010s levels easily without a deeper slowdown.

![UNRATE vs JOLTS](charts/10_unrate_vs_jolts.png)

Housing shows how policy transmits: mortgage rates choke construction before they bend services CPI.

![Housing vs Mortgage](charts/11_housing_vs_mortgage.png)

---

## 9) The Inflation Mix: Core PCE vs CPI

Core PCE better maps to the Fed’s mandate; both series say the **floor** is higher than
the 2010s median.

![Core PCE vs CPI](charts/12_corepce_vs_cpi.png)

---

## 10) Financial Conditions and Credit Risk

Financial conditions are not restrictive enough to guarantee a rapid return to 2%.

![NFCI](charts/13_nfci.png)

Credit spreads haven’t blown out — yet. But a higher real rate world leaves less cushion.

![BAA minus 10Y](charts/14_baa_minus_10y.png)

---

## 11) Growth Pulse: Payrolls vs Production

Employment growth is slowing but not collapsing; **industrial production** is the more
cyclical read-through. The “soft landing” is plausible — **but with a higher inflation floor**.

![Payrolls vs Industrial Production](charts/15_payrolls_vs_indpro.png)

---

## Bottom Line

- **Structural inflation** is a policy problem with geopolitical, demographic, and energy roots.  
- **Term premium** is back; curves won’t behave like 2010–2019.  
- **Positioning implication (non-advice):** assets that breathe with real rates, cash flows, and scarcity
  should command attention.

*This is general information only and not financial advice. For personal guidance, please talk to a licensed professional.*